**NOTE:** You don't need to !pip install within the notebooks themselves. If you need any packages, just add them to the requirements.txt file, and Deepnote will automaticaly grab them whenever its starts the machine.

In [1]:
#Installing necessary packages
!pip install geopandas

#Installing needed geo based libraries
!pip install libpysal
!pip install splot
!apt update
!apt upgrade
!sudo apt-get update
!sudo apt install -y gdal-bin python-gdal python3-gdal 

# Install rtree - Geopandas requirment
!sudo apt install -y python3-rtree 
!pip install pysal

You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
Reading package lists... Done
E: List directory /var/lib/apt/lists/partial is missing. - Acquire (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock - open (13: Permission denied)
E: Unable to lock the administration directory (/var/lib/dpkg/), are you root?
Ign:1 http://deb.debian.org/debian stretch InRelease
Get:2 http://security.debian.org/debian-security stretch/updates InRelease [53.0 kB]
Get:3 http://deb.debian.org/debian stretch-updates InRelease [93.6 kB]
Get:4 http://deb.debian.org/debian stretch Release [118 kB]           
Get:5 http://deb.debian.org/debian stretch Release.gpg [2,410 B]
Get:6 http://security.debian.org/debian-security stretch/updates/main amd64 Pack

In [ ]:
import geopandas as gpd
import fiona
from shapely.geometry import shape, mapping
import rtree
from rtree import index
import fiona.crs

In [ ]:
la_redline = gpd.read_file("/home/jovyan/work/COVIDRedlining/data/la/redlining_la/cartodb-query.shp", skiprows = 1)
la_ct = gpd.read_file("/home/jovyan/work/COVIDRedlining/data/la/census_tracts/tl_2019_06_tract.shp", skiprows = 1)

In [ ]:
la_ct.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,06,037,139301,06037139301,1393.01,Census Tract 1393.01,G5020,S,2865657,0,+34.1781538,-118.5581265,"POLYGON ((-118.57150 34.17758, -118.57148 34.1..."
1,06,037,139302,06037139302,1393.02,Census Tract 1393.02,G5020,S,338289,0,+34.1767230,-118.5383655,"POLYGON ((-118.54073 34.18019, -118.54070 34.1..."
2,06,037,139502,06037139502,1395.02,Census Tract 1395.02,G5020,S,1047548,0,+34.1628402,-118.5263110,"POLYGON ((-118.53225 34.16201, -118.53177 34.1..."
3,06,037,139600,06037139600,1396,Census Tract 1396,G5020,S,2477482,0,+34.1640599,-118.5101001,"POLYGON ((-118.51858 34.15858, -118.51858 34.1..."
4,06,037,139701,06037139701,1397.01,Census Tract 1397.01,G5020,S,3396396,2411,+34.1574290,-118.4954117,"POLYGON ((-118.50980 34.15691, -118.50848 34.1..."


In [ ]:
rla_fiona = fiona.open('/home/jovyan/work/COVIDRedlining/data/la/redlining_la/cartodb-query.shp')
ctla_fiona = fiona.open('/home/jovyan/work/COVIDRedlining/data/la/census_tracts/tl_2019_06_tract.shp')

In [ ]:
fc_intersect = []

for featA in ctla_fiona:
    geomA = featA['geometry']

    for featB in rla_fiona:
        geomB = featB['geometry']

        if shape(geomA).intersects(shape(geomB)):
            fc_intersect.append([geomA,geomB])